In [151]:
import requests
import json
import os
import pandas as pd
from functools import reduce
import operator
import pyjsonviewer
import geopandas as gpd
import time
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Point
from pymongo import MongoClient
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from pymongo import GEOSPHERE
import shapely.geometry

from dotenv import load_dotenv
load_dotenv()

True

## Funciones útiles

In [14]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [15]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [16]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [data["latt"], data["longt"]]}
    except:
        return data

In [414]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
geospatial = db.get_collection("geospatial")

## Lista de categorías e id de Foursquare
- scrapping

In [17]:
def get_categories():
    url = "https://developer.foursquare.com/docs/build-with-foursquare/categories/"
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    venue_cat = soup.find_all("ul", {"class": "VenueCategories__Wrapper-sc-15dn453-0 dmcDKQ"})
    cat = {}
    li = venue_cat[0].find_all("li")
    for l in li:
        cat[l.find("h3").getText()] = l.find("p").getText()
        
    return pd.DataFrame.from_dict(cat,orient="Index").reset_index().rename(columns={"index": "category", 0: "categoryId"})

categories = get_categories()

In [18]:
categories.sample(3)

,category,categoryId
297,Ouzeri,53d6c1b0e4b02351e88a83dc
304,Poke Place,5bae9231bedf3950379f89d4
11,Disc Golf,52e81612bcbc57f1066b79e8


In [19]:
def categories_dic(cat_list):    
    dicc = {}
    for c in cat_list:
        dicc[c] = categories.categoryId[categories["category"] == c].values[0]
    return dicc

In [20]:
venues_list = ["Design Studio","Nursery School","Preschool","Elementary School","Tech Startup","Coffee Shop",
               "Airport Terminal","Train Station",
              "Nightlife Spot","Vegetarian / Vegan Restaurant","Basketball Stadium",
               "Veterinarian","Pet Service","Pet Store"]  

In [21]:
cat_dic = categories_dic(venues_list)
cat_dic

{'Design Studio': '4bf58dd8d48988d1f4941735',
 'Nursery School': '4f4533814b9074f6e4fb0107',
 'Preschool': '52e81612bcbc57f1066b7a45',
 'Elementary School': '4f4533804b9074f6e4fb0105',
 'Tech Startup': '4bf58dd8d48988d125941735',
 'Coffee Shop': '4bf58dd8d48988d1e0931735',
 'Airport Terminal': '4bf58dd8d48988d1eb931735',
 'Train Station': '4bf58dd8d48988d129951735',
 'Nightlife Spot': '4d4b7105d754a06376d81259',
 'Vegetarian / Vegan Restaurant': '4bf58dd8d48988d1d3941735',
 'Basketball Stadium': '4bf58dd8d48988d18b941735',
 'Veterinarian': '4d954af4a243a5684765b473',
 'Pet Service': '5032897c91d4c4b30a586d69',
 'Pet Store': '4bf58dd8d48988d100951735'}

In [22]:
def localizations_coor(dicc):
    local = {}
    for city,loc in localizations_dicc.items():
        local[city] = geocode(loc)["coordinates"]
        time.sleep(15)
    return local
    

In [23]:
# localizations_dicc={"Austin":"Texas Capitol, Austin, Texas","Hamburg":"Hamburg city center,Hamburg,Germany","Newcastle":"Newcastle upon Tyne, United Kingdom"}
# localizations = localizations_coor(localizations_dicc)
# localizations

In [24]:
 localizations = {'Austin': ['30.27125853189173', '-97.75551842126049'],
 'Hamburg': ['53.55687309076416', '10.002900693677232'],
 'Newcastle': ['54.96923892979405', '-1.6162802662146134']}

In [25]:
def place_store_dicc(localizations,cat_dic):
    url_query = 'https://api.foursquare.com/v2/venues/search'
    client_id = os.getenv("four")
    client_secret = os.getenv("foursec")
    
    venues_dic = {}
    aus_dic ={}
    ham_dic = {}
    ncas_dic ={}
    for loc,coor in localizations.items():
        for cat,cat_id in cat_dic.items():
            time.sleep(1)
            parametros={
                "client_id": client_id,
                "client_secret": client_secret,
                "v": "20180323",
                "ll": f"{coor[0]}, {coor[1]}",
                "categoryId":cat_id,
                "radius":2000
            }
            if loc == "Austin":
                aus_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"] 
                if cat == "Pet Store":
                    print(f"{loc} done")
            elif loc == "Hamburg":                
                ham_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"]
                if cat == "Pet Store":
                    print(f"{loc} done")
            elif loc == "Newcastle":                
                ncas_dic[cat] = requests.get(url_query, params = parametros).json()["response"]["venues"] 
                if cat == "Pet Store":
                    print(f"{loc} done")
            
    venues_dic = {"Austin":aus_dic,
                 "Hamburg":ham_dic,
                  "Newcastle": ncas_dic
                 }

    return venues_dic


In [26]:
total_venues = place_store_dicc(localizations,cat_dic)

Austin done
Hamburg done
Newcastle done


In [49]:
with open('../data/total_venues.json', 'w') as f:
    json.dump(total_venues, f)

In [35]:
def extraetodo(json):
    total = []
    for city,categories in json.items():
        for cat,value in categories.items():
            try:
                todo = {"nombre": ["name"],"sub_category":["categories",0,"name"],
                        "latitud": ["location", "lat"], "longitud": ["location", "lng"],
                        "distance":["location","distance"]}

                for elemento in value:
                    store = {key: getFromDict(elemento,value) for key,value in todo.items()}
                    store["location"] = type_point([store["latitud"], store["longitud"]])
                    store["city"] = city
                    store["category"] = cat
                    total.append(store)

                #print(f"{elemento} done")
            except:

                print(f"{value.index(elemento)} wrong")

    return total

In [305]:
venues = extraetodo(total_venues)

In [397]:
venues_df = pd.DataFrame(venues)

In [431]:
venues_df.category.unique()

array(['Design Studio', 'School', 'Tech Startup', 'Coffee Shop',
       'Train Station', 'Nightlife Spot', 'Vegetarian / Vegan Restaurant',
       'Basketball Stadium', 'Dobby'], dtype=object)

In [404]:
venues_df.category = venues_df.category.str.replace(r"(.*[Ss](chool|CHOOL).*)","School",regex=True)
venues_df.category = venues_df.category.str.replace(r"(.*[Pp](et|ET)\s.*|.*[Vv](eterinarian|ETERINARIAN).*)","Dobby",regex=True)#

In [405]:
#borrar todos los cafés que no sean starbucks
no_starbucks = list(venues_df.index[(venues_df["category"] == "Coffee Shop") & (venues_df["nombre"] != "Starbucks")])
venues_df.drop(no_starbucks,axis=0, inplace = True)

In [406]:
venues_df.reset_index(drop = True,inplace = True)

In [407]:
venues_df.to_csv("../data/venues_df.csv",index = False)

In [430]:
# Austin_df = venues_df[venues_df["city"] == "Austin"].sort_values(["category","distance"])
# Hamburg_df = venues_df[venues_df["city"] == "Hamburg"].sort_values(["category","distance"])
# Newcastle_df = venues_df[venues_df["city"] == "Newcastle"].sort_values(["category","distance"])
#Austin_df.groupby("category").agg({"distance":"mean"}).reset_index().head(3)
#Hamburg_df.groupby("category").agg({"distance":"mean"}).reset_index().head(3)
#Newcastle_df.groupby("category").agg({"distance":"mean"}).reset_index().head(3)

In [122]:
# with open('../data/venues_Austin.json', 'w') as f:
#     json.dump(venues_Austin, f)

In [437]:
geo_venues = gpd.GeoDataFrame(venues_df, geometry=gpd.points_from_xy(venues_df.longitud,venues_df.latitud))
geo_venues.drop("location", axis=1, inplace = True)
geo_venues.head(2)

,nombre,sub_category,latitud,longitud,distance,city,category,geometry
0,Urbanspace Interiors,Design Studio,30.269045,-97.75153,455,Austin,Design Studio,POINT (-97.75153 30.26905)
1,Fjord Austin,Design Studio,30.265794,-97.74359,1298,Austin,Design Studio,POINT (-97.74359 30.26579)


In [438]:
geo_venues['geometry']=geo_venues['geometry'].apply(lambda x:shapely.geometry.mapping(x))
geo_venus_dic = geo_venues.to_dict(orient="records")
geospatial.create_index([("geometry", GEOSPHERE)])
geospatial.insert_many(geo_venus_dic)

/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [439]:
aus_coor = [-97.75551842126049,30.27125853189173] #longitud / latitud
Ham_coor =[10.002900693677232, 53.55687309076416]
Ncas_coor =  [-1.6162802662146134 ,54.96923892979405]

In [440]:
coor = type_point(aus_coor)
metros = 2000
query = {"geometry": {"$near": {"$geometry": coor, "$maxDistance": metros}}}
list(geospatial.find(query))

[{'_id': ObjectId('61926245ae2bab83bfad4129'),
  'nombre': 'Pixel Peach',
  'sub_category': 'Design Studio',
  'latitud': 30.270267,
  'longitud': -97.75677,
  'distance': 163,
  'city': 'Austin',
  'category': 'Design Studio',
  'geometry': {'type': 'Point', 'coordinates': [-97.75677, 30.270267]}},
 {'_id': ObjectId('61926245ae2bab83bfad4185'),
  'nombre': 'Juiceland',
  'sub_category': 'Juice Bar',
  'latitud': 30.270983,
  'longitud': -97.753333,
  'distance': 212,
  'city': 'Austin',
  'category': 'Vegetarian / Vegan Restaurant',
  'geometry': {'type': 'Point', 'coordinates': [-97.753333, 30.270983]}},
 {'_id': ObjectId('61926245ae2bab83bfad413f'),
  'nombre': 'Qcue Headquarters',
  'sub_category': 'Tech Startup',
  'latitud': 30.272009000000004,
  'longitud': -97.757663,
  'distance': 222,
  'city': 'Austin',
  'category': 'Tech Startup',
  'geometry': {'type': 'Point',
   'coordinates': [-97.757663, 30.272009000000004]}},
 {'_id': ObjectId('61926245ae2bab83bfad415c'),
  'nombre':